In [ ]:
from avalanche.benchmarks import SplitMNIST

benchmark = SplitMNIST(n_experiences=2, seed=42)

train_stream = benchmark.train_stream
test_stream = benchmark.test_stream

for experience in train_stream: 
    print("Start of task ", experience.task_label)
    print('Classes in this task:', experience.classes_in_this_experience)

    current_training_set = experience.dataset
    print('Task {}'.format(experience.task_label))
    print('This task contains', len(current_training_set), 'training examples')

    current_test_set = test_stream[experience.current_experience].dataset
    print('This task contains', len(current_test_set), 'test examples')


Start of task  0
Classes in this task: [1, 2, 4, 6, 8]
Task 0
This task contains 30311 training examples
This task contains 5081 test examples
Start of task  0
Classes in this task: [0, 3, 5, 7, 9]
Task 0
This task contains 29689 training examples
This task contains 4919 test examples


Defining the file logger

In [20]:
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, \
    accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
    confusion_matrix_metrics, disk_usage_metrics, StreamConfusionMatrix
from avalanche.logging import InteractiveLogger

text_logger = InteractiveLogger()
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    StreamConfusionMatrix(num_classes=benchmark.n_classes, save_image=False),
    loggers=[text_logger]
)

Models and Strategies

In [25]:
from avalanche.models import SimpleMLP
from avalanche.training import Naive, EWC
from torch.optim import SGD, RMSprop
from torch.nn import CrossEntropyLoss
from avalanche.training.plugins import EWCPlugin


import json

# Load hyperparameters from JSON
with open('baseline_config.json', 'r') as f:
    hyperparams = json.load(f)


model = SimpleMLP(num_classes=benchmark.n_classes,hidden_layers=2)

# Set up the optimizer using loaded hyperparameters
optimizer = RMSprop(
    model.parameters(),
    lr=hyperparams.get("learning_rate"),
    momentum=hyperparams.get("momentum"),
    alpha=hyperparams.get("decay")
)

shared_params = {
    "model": model,
    "optimizer": optimizer,
    "criterion": CrossEntropyLoss(),
    "train_mb_size": hyperparams.get("minibatch_size"),
    "train_epochs": 20,
    "eval_mb_size": 200,
    "evaluator": eval_plugin
}

ewc_params = {
    "ewc_lambda": hyperparams.get("Fisher_multiplier"),
    #"plugins": [EWCPlugin(ewc_lambda=hyperparams.get("Fisher_multiplier", 400))]
}

naive_strategy = Naive(**shared_params)
ewc_strategy = EWC(**shared_params, **ewc_params)



Training

In [22]:
results_naive = []
print('Starting experiment...')

for exp_id, experience in enumerate(train_stream):
    print("Start of experience ", experience.current_experience)

    naive_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    results_naive.append(naive_strategy.eval(benchmark.test_stream[exp_id]))

print('Final evaluation...')
naive_strategy.eval(benchmark.test_stream)

Starting experiment...
Start of experience  0
-- >> Start of training phase << --
0it [00:00, ?it/s]

100%|██████████| 948/948 [00:07<00:00, 129.94it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1863
	Loss_MB/train_phase/train_stream/Task000 = 0.0061
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9428
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 948/948 [00:07<00:00, 133.97it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0858
	Loss_MB/train_phase/train_stream/Task000 = 0.0213
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9738
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 948/948 [00:07<00:00, 135.04it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0662
	Loss_MB/train_phase/train_stream/Task000 = 0.0002
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9811
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 948/948 [00:06<00:00, 146.71it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0536
	Loss_MB/train_phase/train_s

{'Top1_Acc_MB/train_phase/train_stream/Task000': 1.0,
 'Loss_MB/train_phase/train_stream/Task000': 0.00026931148022413254,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9921182929704604,
 'Loss_Epoch/train_phase/train_stream/Task000': 0.03403221873003058,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.0,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 107.25927518902039,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.4858,
 'Loss_Stream/eval_phase/test_stream/Task000': 54.53128669113871,
 'StreamForgetting/eval_phase/test_stream': 0.9899626057862625,
 'ConfusionMatrix_Stream/eval_phase/test_stream': tensor([[ 974,    0,    0,    1,    0,    2,    0,    1,    0,    2],
         [   0,    0,    0,  288,    0,   12,    0,  826,    0,    9],
         [  73,    0,    0,  788,    0,    3,    0,  159,    0,    9],
         [   0,    0,    0,  997,    0,    3,    0,    4,    0,    6],
         [  17,    0,    0,    2,    0,    2,    0,   45,    0,  916],
         [  

In [26]:
results_ewc = []
print('Starting experiment...')

for exp_id, experience in enumerate(train_stream):
    print("Start of experience ", experience.current_experience)

    ewc_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    results_ewc.append(ewc_strategy.eval(benchmark.test_stream[exp_id]))

print('Final evaluation...')
ewc_strategy.eval(benchmark.test_stream)

Starting experiment...
Start of experience  0
-- >> Start of training phase << --
100%|██████████| 948/948 [15:11<00:00,  1.04it/s]    
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1851
	Loss_MB/train_phase/train_stream/Task000 = 0.0068
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9426
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 948/948 [00:06<00:00, 152.80it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0889
	Loss_MB/train_phase/train_stream/Task000 = 0.0014
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9744
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 948/948 [00:06<00:00, 148.50it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0651
	Loss_MB/train_phase/train_stream/Task000 = 0.0003
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9814
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 948/948 [00:06<00:00, 146.87it/s]
Epoch 3 ende

{'Top1_Acc_MB/train_phase/train_stream/Task000': 1.0,
 'Loss_MB/train_phase/train_stream/Task000': 0.0120717603713274,
 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9833271582067432,
 'Loss_Epoch/train_phase/train_stream/Task000': 0.076755966987861,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.02775044282621531,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 17.174545127795078,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.4999,
 'Loss_Stream/eval_phase/test_stream/Task000': 8.756970561776887,
 'StreamForgetting/eval_phase/test_stream': 0.9639834678212951,
 'ConfusionMatrix_Stream/eval_phase/test_stream': tensor([[ 974,    0,    0,    1,    0,    4,    0,    1,    0,    0],
         [   9,    0,    0,  178,    0,   30,    0,  901,    0,   17],
         [  95,    0,   61,  622,    0,    3,    0,  248,    0,    3],
         [   1,    0,    0,  990,    0,    9,    0,    7,    0,    3],
         [  19,    0,    0,    1,    1,    8,    0,  104,    0,  849],
  